In [2]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random

headers = { 
	"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36", 
	"Referer": "https://www.domain.com.au/"
}

base_url = "https://www.domain.com.au/rent/"
errCount = 0
listing_ids = []

for i in range(1,6):
    if i > 1:
        url = f"{base_url}?page={i}"
    else:
        url = base_url
    
    response = requests.get(url, headers=headers)
    if response.ok:
        soup = BeautifulSoup(response.content, 'html')
        elements_with_class = soup.find_all(class_='css-1qp9106')
        
        for element in elements_with_class:
            data_testid_value = element.get('data-testid')
            if data_testid_value:
                listing_id = re.sub(r'\D', '', data_testid_value)
                listing_ids.append(listing_id)
        time.sleep(random.random()+0.3)

    else:
        print(response)
        errCount += 1
        time.sleep(random.random()+0.3)


In [5]:
headers = {
    'X-API-Key': 'key_07e6c61e2af44856103c7a58aa5339aa'
    }

base_url = 'https://api.domain.com.au/v1/me'

authenticate = requests.get(base_url, headers=headers)

authenticate.status_code


200

In [6]:
import pandas as pd

base_url = 'https://api.domain.com.au/v1/listings/'
url = f"{base_url}{listing_ids[0]}"
response = requests.get(url, headers=headers)

if response.ok:
    listing = response.json()
else:
    raise RuntimeError

titles = list(listing.keys())

df = pd.DataFrame(columns = titles)


In [7]:
df = df.drop(columns=['geoLocation', 'isNewDevelopment', 'media', 'channel', 
                      'rentalDetails', 'advertiserIdentifiers', 'inspectionDetails', 
                      'objective', 'saleMode', 'description'])

In [8]:
for i in listing_ids:
    url = f'{base_url}{i}'
    response = requests.get(url, headers=headers)
    if response.ok:
        listing = response.json()
        df.loc[len(df.index)] = listing

In [9]:
df['addressParts'] = df['addressParts'].apply(lambda x: x['displayAddress'])
df = df.rename(columns={'addressParts': 'address'})

df['apmIdentifiers'] = df['apmIdentifiers'].apply(lambda x: x['suburbId'])
df = df.rename(columns = {'apmIdentifiers': 'suburbId'})

In [10]:
df

,status,id,address,suburbId,bathrooms,bedrooms,carspaces,dateAvailable,dateUpdated,dateListed,features,headline,landAreaSqm,priceDetails,propertyId,propertyTypes,seoUrl
0,new,16838698,"21 Hastings Street, Annerley QLD 4103",914,1,3,1,2024-02-02T00:00:00+11:00,2024-01-18T06:25:27.593Z,2024-01-18T06:20:48Z,"[Air conditioning, Built in wardrobes, Balcony...",PRIVATE INNER CITY RETREAT,410.0,"{'price': 770, 'canDisplayPrice': True, 'displ...",QN-1250-NU,[house],https://www.domain.com.au/21-hastings-street-a...
1,new,16839479,"108/11 Garrigarrang Avenue, Kogarah NSW 2217",24012,1,1,1,2024-02-02T00:00:00+11:00,2024-01-18T06:16:26.443Z,2024-01-18T06:16:26Z,NaN,Lifestyle Home,NaN,"{'canDisplayPrice': False, 'displayPrice': '$6...",UX-7491-TX,[house],https://www.domain.com.au/108-11-garrigarrang-...
2,new,16839471,"1/12 Potter Street, South Toowoomba QLD 4350",33794,1,2,1,2024-02-13T00:00:00+11:00,2024-01-18T06:15:39.16Z,2024-01-18T06:15:39Z,"[Broadband internet access, Built in wardrobes...",Quiet and Secure South Toowoomba Unit - Ready Now,NaN,"{'canDisplayPrice': False, 'displayPrice': '$3...",GB-9898-BM,[apartmentUnitFlat],https://www.domain.com.au/1-12-potter-street-s...
3,new,16839453,"9/9 Carter Street, North Ward QLD 4810",28524,2,2,1,2024-02-01T00:00:00+11:00,2024-01-18T06:10:29.57Z,2024-01-18T06:10:29Z,NaN,One Block from the Strand!,NaN,"{'canDisplayPrice': False, 'displayPrice': '$4...",ZV-5980-MR,[apartmentUnitFlat],https://www.domain.com.au/9-9-carter-street-no...
4,new,16839445,"2 Hazeldine Road, Glen Iris VIC 3146",15907,2,4,2,2024-01-29T00:00:00+11:00,2024-01-18T06:08:38.313Z,2024-01-18T06:08:38Z,NaN,Lovely home in prime location,636.0,"{'canDisplayPrice': False, 'displayPrice': '$1...",MG-0026-FK,[house],https://www.domain.com.au/2-hazeldine-road-gle...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,new,16839046,"16 Endeavour Avenue, Orange NSW 2800",34392,1,3,1,2024-01-22T00:00:00+11:00,2024-01-18T04:26:11.327Z,2024-01-18T04:26:11Z,NaN,UPDATED THREE BEDROOM HOME IN A GREAT LOCATION,NaN,"{'canDisplayPrice': False, 'displayPrice': '$5...",MQ-5356-FL,[house],https://www.domain.com.au/16-endeavour-avenue-...
96,new,16839043,"4/45 Todman Avenue, Kensington NSW 2033",23092,1,2,0,2024-01-22T00:00:00+11:00,2024-01-18T04:51:50.483Z,2024-01-18T04:25:47Z,"[Bathtub, Balcony]","CONTEMPORARY TOP FLOOR, TWO BEDROOM APARTMENT ...",NaN,"{'price': 750, 'canDisplayPrice': True, 'displ...",KX-5924-GP,[apartmentUnitFlat],https://www.domain.com.au/4-45-todman-avenue-k...
97,new,16839039,"39 General Holmes Drive, Brighton-Le-Sands NSW...",6332,2,2,1,2024-01-19T00:00:00+11:00,2024-01-18T05:08:50.837Z,2024-01-18T04:25:11Z,NaN,2 BEDROOM HOME.,NaN,"{'canDisplayPrice': False, 'displayPrice': '$7...",LQ-3083-LO,[house],https://www.domain.com.au/39-general-holmes-dr...
98,recentlyUpdated,15171920,"6 Highfield Street, Mayfield NSW 2304",27862,8,1,0,2024-02-01T00:00:00+11:00,2024-01-18T04:25:07.167Z,2024-01-18T04:25:07Z,"[Air Conditioning, Built-In Wardrobes, Close t...",Your Next Co-Living Move.,NaN,"{'canDisplayPrice': False, 'displayPrice': '$2...",BC-0729-QV,[house],https://www.domain.com.au/6-highfield-street-m...


In [42]:
from minio import Minio
# Initialize MinIO client
client = Minio('localhost:9000',
               access_key='accessKey',
               secret_key='secretKey',
               secure=False)
# Make a bucket

df.to_csv(r'./output.csv')

client.fput_object('domain-data', 'output.csv', 'output.csv')


In [1]:
client = Minio('localhost:9000',
               access_key='accessKey',
               secret_key='secretKey',
               secure=False)

client.list_buckets()

NameError: name 'Minio' is not defined